In [73]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
#just for the notebook
%run implementations
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [44]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
DATA_TEST_PATH = '../data/test.csv'
y, tX, train_ids = load_csv_data(DATA_TRAIN_PATH)
subsmission_y, submission_x, test_ids = load_csv_data(DATA_TEST_PATH)

In [66]:
DEGREE = 3

In [46]:
x_train, y_train, x_test, y_test = split_data(tX, y, 0.8, seed=1)

In [47]:
# Append y values as column to later divide y into buckets corresponding with x values
x_train = np.column_stack((x_train, y_train))
x_test = np.column_stack((x_test, y_test))

In [56]:
# get all the needed buckets
buckets = get_buckets(x_train)
# y
y_buckets = []
for i in range(len(buckets)):
    y_buckets.append(buckets[i][:, -1])
    buckets[i] = np.delete(buckets[i], -1, 1)
# test
test_buckets = get_buckets(x_test)
submission_x_buckets = get_buckets(submission_x.copy())
# test y
test_y_buckets = []
for i in range(len(test_buckets)):
    test_y_buckets.append(test_buckets[i][:, -1])
    test_buckets[i] = np.delete(test_buckets[i], -1, 1)

In [49]:
# Normalize the matrices
for i in range(len(buckets)):
    buckets[i] = normalize(buckets[i])
    test_buckets[i] = normalize(test_buckets[i])
    submission_x_buckets[i] = normalize(submission_x_buckets[i])

In [50]:
weights = []
for i in range(len(buckets)):
    w = np.zeros((buckets[i].shape[1], 1))
    w_rr, loss_rr = logistic_regression(y=y_buckets[i], tx=buckets[i], initial_w=w,
                                        max_iters=100, gamma=0.01)
    weights.append(w_rr)

MemoryError: 

In [91]:
weights = []
for i in range(len(buckets)):
    w = np.zeros((buckets[i].shape[1], 1))
    w_rr, loss_rr = ridge_regression(y=y_buckets[i], tx=buckets[i], lambda_=0.7)
    weights.append(w_rr)

In [89]:
def accuracy(train_buckets, test_buckets, test_y_buckets, weights, degree, lambda_):
    correct_predictions = 0
    len_data = 0
    for i in range(len(train_buckets)):
        rr_accuracy = compute_accuracy(weights[i], test_buckets[i], test_y_buckets[i])
        correct_predictions += (rr_accuracy * len(test_buckets[i]))
        len_data += len(test_buckets[i])
    total_accuracy = correct_predictions / len_data
    print(f"Accuracy = {total_accuracy} with degree = {degree} and lambda = {lambda_}")
    return total_accuracy

In [93]:
acc = accuracy(train_buckets=buckets, test_buckets=test_buckets, test_y_buckets=test_y_buckets,
                            weights=weights, degree=3, lambda_=0.7)

Accuracy = 0.76372 with degree = 3 and lambda = 0.7


## Do your thing crazy machine learning thing here :) ...

## Generate predictions and save ouput in csv format for submission:

In [94]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = '../data/submission.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)